In [2]:
import os
import xarray as xr
import pandas as pd
import datetime
import numpy as np

## Load ISMIP Variables

In [3]:
ismip  = pd.read_csv('/mnt/d/1_protect/0_sanity_check/ismip_variable.csv',delimiter=';')
ismip.head()

,variable,variable_alias,dim,type,name,standard_name,alias_name,units,min_value,max_value
0,lithk,NaN,"x,y,t",ST,Ice thickness,land_ice_thickness,NaN,m,0,1000
1,orog,NaN,"x,y,t",ST,Surface elevation,surface_altitude,NaN,m,0,1000
2,base,NaN,"x,y,t",ST,Base elevation,base_altitude,NaN,m,0,1000
3,topg,NaN,"x,y,t",ST,Bedrock elevation,bedrock_altitude,NaN,m,0,1000
4,hfgeoubed,NaN,"x,y,t",FL,Geothermal heat flux,upward_geothermal_heat_flux_in_land_ice,upward_geothermal_heat_flux_at_ground_level,W m-2,0,1000


In [4]:
ismip_meta = ismip.to_dict('records')
ismip_meta

[{'variable': 'lithk',
  'variable_alias': nan,
  'dim': 'x,y,t',
  'type': 'ST',
  'name': 'Ice thickness',
  'standard_name': 'land_ice_thickness',
  'alias_name': nan,
  'units': 'm',
  'min_value': 0,
  'max_value': 1000},
 {'variable': 'orog',
  'variable_alias': nan,
  'dim': 'x,y,t',
  'type': 'ST',
  'name': 'Surface elevation',
  'standard_name': 'surface_altitude',
  'alias_name': nan,
  'units': 'm',
  'min_value': 0,
  'max_value': 1000},
 {'variable': 'base',
  'variable_alias': nan,
  'dim': 'x,y,t',
  'type': 'ST',
  'name': 'Base elevation',
  'standard_name': 'base_altitude',
  'alias_name': nan,
  'units': 'm',
  'min_value': 0,
  'max_value': 1000},
 {'variable': 'topg',
  'variable_alias': nan,
  'dim': 'x,y,t',
  'type': 'ST',
  'name': 'Bedrock elevation',
  'standard_name': 'bedrock_altitude',
  'alias_name': nan,
  'units': 'm',
  'min_value': 0,
  'max_value': 1000},
 {'variable': 'hfgeoubed',
  'variable_alias': nan,
  'dim': 'x,y,t',
  'type': 'FL',
  'name':

In [5]:
# get the list of variables
ismip_var = [dic['variable'] for dic in ismip_meta]

## Explore directory

In [6]:
source_path = '/mnt/d/1_protect/0_sanity_check/IMAU/IMAUICE1'

In [63]:
def list_files(path):
    for root,dirs,files in os.walk(path):
        level = root.replace(path,'').count(os.sep)
        indent = ' ' * 4 *(level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = '  '*4*(level+1)
        for f in files:
            print(f'{subindent}{f}')

#list_files(source_path)

In [65]:
def files_and_subdirectories(root_path):
    files = []
    directories = []
    for f in os.listdir(root_path):
        if os.path.isfile(f):
            files.append(f)
        elif os.path.isdir(f):
            directories.append(f)
    return directories, files

directories,files = files_and_subdirectories(source_path)

print(directories)


[]


## Check a single file

In [68]:
#file = '/mnt/d/1_protect/0_sanity_check/IMAU/IMAUICE1/exp05_32/acabf_AIS_IMAU_IMAUICE1_exp05.nc'
#!scalar! file = '/mnt/d/1_protect/0_sanity_check/IMAU/IMAUICE1/abmb_32/iareafl_AIS_IMAU_IMAUICE1_abmb.nc'
file = '/mnt/d/1_protect/0_sanity_check/IMAU/IMAUICE1/exp05_32/libmassbffl_AIS_IMAU_IMAUICE1_exp05.nc'
#file = '/mnt/d/1_protect/0_sanity_check/IMAU/IMAUICE1/exp05_32/orog_AIS_IMAU_IMAUICE1_exp05.nc'
ds = xr.open_dataset(file)
ds

<xarray.Dataset>
Dimensions:      (time: 86, y: 191, x: 191, nv: 2)
Coordinates:
  * time         (time) datetime64[ns] 2015-07-01 2016-07-01 ... 2100-07-01
  * x            (x) float32 -3.04e+06 -3.008e+06 ... 3.008e+06 3.04e+06
  * y            (y) float32 -3.04e+06 -3.008e+06 ... 3.008e+06 3.04e+06
Dimensions without coordinates: nv
Data variables:
    libmassbffl  (time, y, x) float32 ...
    mapping      |S1 ...
    time_bounds  (time, nv) datetime64[ns] ...
Attributes:
    NCO:                        netCDF Operators version 4.7.7 (Homepage = ht...
    Conventions:                CF-1.7
    info:                       Model IMAUICE1, Institute for Marine and Atmo...
    contact:                    Heiko Goelzer, h.goelzer@uu.nl 
    history:                    Thu Oct 17 20:34:52 2019: ncks -A -v time,tim...
    history_of_appended_files:  Thu Oct 17 20:34:52 2019: Appended file ../Da...

In [69]:
file_var = list(ds.data_vars)
errors = 0
warnings = 0

for ivar in file_var:
    if ivar in ismip_var:
        print('* ',ivar, 'is a valid name.')
        # get index in the ismip_var list
        var_index = [k for k in range(len(ismip_var)) if ismip_var[k]==ivar]
        # check the unit
        if ds[ivar].attrs['units'] == ismip_meta[var_index[0]]['units']:
            print('the unit is correct:',ds[ivar].attrs['units'])

        else:
            print('the current variable\'s unit is',ds[ivar].attrs['units'],'and should be',ismip_meta[var_index[0]]['units'],'Please check.')
            warnings = warnings + 1 
        # check the min,max,mean values
        if ds[ivar].min()>ismip_meta[var_index[0]]['min_value']:
            print('The minimum value successfully verified.')
        else:
            print('The minimum value(', round(ds[ivar].min().values.item(0),5),') is out of range. Please check it!')
            errors = errors + 1 
        if ds[ivar].max()>ismip_meta[var_index[0]]['max_value']:
                print('The maximum value successfully verified.')
        else:
            print('The maximum value(', round(ds[ivar].max().values.item(0),5),') is out of range. Please check it!')
            errors = errors + 1

        #SPATIAL: Check spatial extent of the grid
        coords = ds.coords.to_dataset()
        Xbottomleft=int(min(coords['x']).values.item())
        Ybottomleft=int(min(coords['y']).values.item())
        Xtopright=int(max(coords['x']).values.item())
        Ytopright=int(max(coords['y']).values.item())

        if Xbottomleft == -3040000 & Ybottomleft == -3040000:
            print( 'Lowest left corner of the grid is well defined.')
        else:    
            print('Lowest left corner of the grid is not well defined. [-3040000,-3040000] Expected')
            errors = errors + 1
        if Xtopright ==3040000 & Ytopright ==3040000:
            print('Upper right corner is well defined.')
        else:    
            print('Upper rigth corner of the grid is not well defined. [3040000,3040000] Expected')
            errors = errors + 1

        #SPATIAL:check the spatial resolution
        spatial_resolution = 32
        Xresolution = (coords['x'][1].values-coords['x'][0].values)/1000
        Yresolution = (coords['y'][1].values-coords['y'][0].values)/1000
        if Xresolution == spatial_resolution and Yresolution == spatial_resolution:
            print('The spatial resolution (grid size) was successfully verified.')
        else:
            print('The spatial resolution ( ', Xresolution,'or',Yresolution,') is different of ',spatial_resolution,' km. Please check it!','\n')
            error = error + 1

        # Time: experiment duration
        start_exp = min(ds['time']).values.astype('datetime64[D]')
        end_exp  = max(ds['time']).values.astype('datetime64[D]')
        print("Start of the experiment:", start_exp.astype('datetime64[D]'))
        print("End of the experiment:", end_exp.astype('datetime64[D]'))

        # Time: time resolution
        time_step = ds['time'].values[11]-ds['time'].values[10]
        print('Time step:',time_step.astype('timedelta64[D]').item().days,'days','\n')

    else:
        print(ivar,'isn\'t a known variable name. Its verification has been ignored.','\n')

print(errors,' critical errors identified in',file,'. It can\'t be shared as it is.')
print(errors,' warnings identified in',file,'. Please, check before sharing.')

*  libmassbffl is a valid name.
the unit is correct: kg m-2 s-1
The minimum value( -0.00037 ) is out of range. Please check it!
The maximum value( 0.00011 ) is out of range. Please check it!
Lowest left corner of the grid is well defined.
Upper right corner is well defined.
The spatial resolution (grid size) was successfully verified.
Start of the experiment: 2015-07-01
End of the experiment: 2100-07-01
Time step: 365 days 

mapping isn't a known variable name. Its verification has been ignored. 

time_bounds isn't a known variable name. Its verification has been ignored. 

2  critical errors identified in /mnt/d/1_protect/0_sanity_check/IMAU/IMAUICE1/exp05_32/libmassbffl_AIS_IMAU_IMAUICE1_exp05.nc . It can't be shared as it is.
2  warnings identified in /mnt/d/1_protect/0_sanity_check/IMAU/IMAUICE1/exp05_32/libmassbffl_AIS_IMAU_IMAUICE1_exp05.nc . Please, check before sharing.


-3040000.0
3040000.0
-3040000.0
3040000.0


In [38]:
ds.coords['y'].shape

(191,)

In [70]:
ds['libmassbffl']

<xarray.DataArray 'libmassbffl' (time: 86, y: 191, x: 191)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       ...,

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2015-07-01 2016-07-01 ... 2100-07-01
  * x        (x) float32 -3.04e+06 -3.008e+06 -2.976e+06 ... 3.008e+06 3.04e+06
  * y        (y) float32 -3.04e+06 -3.008e+06 -2.976e+06 ... 3.008e+06 3.04e+06
Attributes:
    units:          kg m-2 s-1
    standard_name:  land_ice_basal_specific_mass_balance_flux